In [0]:
!pip install -q keras

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras

Using TensorFlow backend.


In [0]:
datadir='/content/drive/My Drive/Project (1)/dataset (1)/train'

In [0]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [0]:
categories=['Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)','Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy','Potato___healthy','Tomato___healthy','Corn_(maize)___Common_rust_','Tomato___Bacterial_spot','Tomato___Early_blight','Potato___Early_blight','Potato___Late_blight']

In [0]:
len(categories)

13

In [0]:
IMAGE_SIZE=(200,200)
BATCH_SIZE=64

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [0]:
datadir_val='/content/drive/My Drive/Project (1)/dataset (1)/val'

In [0]:
datadir_test='/content/drive/My Drive/Project (1)/dataset (1)/test'

In [0]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                     rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(
    datadir_val, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)
do_data_augmentation = True
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest' )
else:
  train_datagen = validation_datagen
  
train_generator = train_datagen.flow_from_directory(
    datadir, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)


Found 236 images belonging to 13 classes.
Found 11159 images belonging to 13 classes.


In [0]:
in_shape=train_generator.image_shape

In [0]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size = (3, 3), activation='relu', input_shape=in_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(13, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
validation_generator.samples


236

In [0]:
train_generator.samples

11159

In [0]:
train_generator.num_classes

13

In [0]:
EPOCHS=15 #@param {type:"integer"}
STEPS_EPOCHS = train_generator.samples//train_generator.batch_size
VALID_STEPS=validation_generator.samples//validation_generator.batch_size
history = model.fit_generator( 
          train_generator,
          steps_per_epoch=STEPS_EPOCHS,
          epochs=EPOCHS,
          validation_data=validation_generator)


Epoch 1/15
174/174 [==============================] - 2768s 16s/step - loss: 1.8999 - acc: 0.5535 - val_loss: 0.5924 - val_acc: 0.8178
Epoch 2/15
174/174 [==============================] - 127s 732ms/step - loss: 0.6943 - acc: 0.7726 - val_loss: 0.5811 - val_acc: 0.8305
Epoch 3/15
174/174 [==============================] - 129s 739ms/step - loss: 0.5456 - acc: 0.8172 - val_loss: 0.3599 - val_acc: 0.8771
Epoch 4/15
174/174 [==============================] - 128s 736ms/step - loss: 0.4806 - acc: 0.8416 - val_loss: 0.4186 - val_acc: 0.8729
Epoch 5/15
174/174 [==============================] - 127s 729ms/step - loss: 0.4518 - acc: 0.8458 - val_loss: 0.3963 - val_acc: 0.8771
Epoch 6/15
174/174 [==============================] - 126s 727ms/step - loss: 0.4262 - acc: 0.8566 - val_loss: 0.3099 - val_acc: 0.8941
Epoch 7/15
174/174 [==============================] - 127s 728ms/step - loss: 0.3969 - acc: 0.8714 - val_loss: 0.3755 - val_acc: 0.8644
Epoch 8/15
174/174 [=============================

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 98, 98, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 614656)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               78676096  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [0]:
model.save('/content/drive/My Drive/appmodel.h5')

In [0]:
datadir_test1='/content/drive/My Drive/Project (1)/dataset (1)/test'

In [0]:
test_datagen1 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [0]:
test_generator1 = test_datagen1.flow_from_directory(
    datadir_test1, 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)


Found 2854 images belonging to 13 classes.


In [0]:
test_generator1.image_shape

(200, 200, 3)

In [0]:
loss, acc = model.evaluate(test_generator1)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

45/45 [==============================] - 771s 17s/step - loss: 0.3027 - acc: 0.9100
Restored model, accuracy: 91.00%


In [0]:
new_model=tf.keras.models.load_model('/content/drive/My Drive/appmodel.h5')
new_model.summary()